In [ ]:
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import plotly.express as px
import pandas as pd
import numpy as np
from datetime import datetime
from statsmodels.tsa.seasonal import seasonal_decompose
import seaborn as sns
import plotly.figure_factory as ff
from keras.models import Sequential
from keras.layers import LSTM,Dropout,Dense


from sklearn.preprocessing import MinMaxScaler
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)


In [ ]:
df = pd.read_pickle('/kaggle/input/bse-stocks-data-15-minute-interval-historical/SIEMENS-15minute-Hist')
df = pd.DataFrame(df)
df['date'] = df['date'].apply(pd.to_datetime)
df.set_index('date',inplace=True)
df.head(3)

In [ ]:
fig = go.Figure(data=[go.Table(
    header=dict(values=list(['date','open','high','low','close','volume']),
                fill_color='cyan',
                align='left'),
    cells=dict(values=[df.index,df.open, df.high, df.low, df.close,df.volume],
               fill_color='yellow',
               align='left'))
])

fig.show()

In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from plotly.graph_objs import Line

fig = make_subplots(rows=5, cols=1,subplot_titles=('Open','High','Low','Close','Volume'))

fig.add_trace(
    Line(x=df.index, y=df.open),
    row=1, col=1
)

fig.add_trace(
    Line(x=df.index, y=df.high),
    row=2, col=1
)

fig.add_trace(
    Line(x=df.index, y=df.low),
    row=3, col=1
)

fig.add_trace(
    go.Line(x=df.index, y=df.close),
    row=4, col=1
)

fig.add_trace(go.Line(x=df.index, y=df.volume),row=5,col=1)

fig.update_layout(height=1400, width=1000, title_text="OHLC Line Plots")

fig.show()


## Visualizing Patterns in the Data

In [ ]:
#only first 5000 values are taken because it was looking very crowded
result = seasonal_decompose(df.close.head(5000), model='additive', period = 30)
fig = go.Figure()  
fig = result.plot()  
fig.set_size_inches(20, 19)

# Candlestick chart for Siemens

In [ ]:
import plotly.graph_objects as go

import pandas as pd
from datetime import datetime


fig = go.Figure(data=[go.Candlestick(x=df.index,
                open=df['open'],
                high=df['high'],
                low=df['low'],
                close=df['close'])])

fig.show()

# Creating Train Test Data

In [ ]:
new_df = pd.DataFrame()
new_df = df['close']
new_df.index = df.index

new_df.head(3)

In [ ]:
scaler=MinMaxScaler(feature_range=(0,1))
final_dataset=new_df.values

train_data=final_dataset[0:20000,]
valid_data=final_dataset[20000:,]

print(train_data.shape,valid_data.shape,final_dataset.shape)
print(final_dataset[0:5])

In [ ]:
train_df = pd.DataFrame()
valid_df = pd.DataFrame()
train_df['Close'] = train_data
train_df.index = new_df[0:20000].index
valid_df['Close'] = valid_data
valid_df.index = new_df[20000:].index

train_df.head()


In [ ]:
scaler=MinMaxScaler(feature_range=(0,1))
scaled_data=scaler.fit_transform(final_dataset.reshape(-1,1))


x_train_data,y_train_data=[],[]
##Assuming that the Closing price on the ith day 
#depends on the closing prices of the previous 60 days
#the 'Y' constraint is the prices on the (i+60)th day
#the 'X' constraints are the prices from (i to i+60 th days)
## We will apply the Log Short Term Memory Model on the X Constraints
for i in range(60,len(train_data)):
    x_train_data.append(scaled_data[i-60:i,0])
    y_train_data.append(scaled_data[i])

x_train_data,y_train_data=np.array(x_train_data),np.array(y_train_data)
x_train_data=np.reshape(x_train_data,(x_train_data.shape[0],x_train_data.shape[1],1))

# Long Short Term Memory Networks(LSTM)

Do you think about everything from scratch. No. You perform the actions based on your past memory. For example if you are reading a newspaper, you understand words because in your past you have read them and they are stored in your memory. If you encounter a new word then it gets stored in your memory newly. So the question is Do you want your model to process everything from scratch? Or you want to make it more intelligent by creating a memory space. Thats when LSTM comes into the game. LSTM which is long short term memory is the type of RNN which can hold memory for longer period of time. They are a good fit for time series preditiction, or forecasting problems.


In [ ]:
lstm_model=Sequential() #The Model
lstm_model.add(LSTM(units=50,return_sequences=True,input_shape=(x_train_data.shape[1],1))) #Layers
lstm_model.add(LSTM(units=50)) #The Layers
lstm_model.add(Dense(1)) #The Layers

inputs_data=new_df[len(new_df)-len(valid_data)-60:].values
inputs_data=inputs_data.reshape(-1,1)
inputs_data=scaler.transform(inputs_data)

lstm_model.compile(loss='mean_squared_error',optimizer='adam')
#Using Adam Optimizer( Combination of RMS Prop and Weighted Mean Gradient Descent)
#Mean Squared Error as the Modelling Metric
lstm_model.fit(x_train_data,y_train_data,epochs=1,batch_size=1,verbose=2)

lstm_model.summary()

In [ ]:
X_test=[]
for i in range(60,inputs_data.shape[0]):
    X_test.append(inputs_data[i-60:i,0])
X_test=np.array(X_test)

X_test=np.reshape(X_test,(X_test.shape[0],X_test.shape[1],1))
predicted_closing_price=lstm_model.predict(X_test)
predicted_closing_price=scaler.inverse_transform(predicted_closing_price)

In [ ]:
valid_df['Predictions']=predicted_closing_price

# Predictions

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=train_df.index,y=train_df['Close'],
                    mode='lines',
                    name='Siemens Train Data'))
fig.add_trace(go.Scatter(x=valid_df.index,y=valid_df['Close'],
                    mode='lines',
                    name='Siemens Valid Data'))
fig.add_trace(go.Scatter(x=valid_df.index,y=valid_df['Predictions'],
                    mode='lines',
                    name='Prediction'))

In [ ]:
from sklearn.metrics import mean_squared_error
print('The Mean Squared Error is',mean_squared_error(valid_df['Close'].values,valid_df['Predictions'].values))

# Let's have a closer look

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=valid_df.index,y=valid_df['Close'],
                    mode='lines',
                    name='Test'))
fig.add_trace(go.Scatter(x=valid_df.index,y=valid_df['Predictions'],
                    mode='lines',
                    name='Predicted'))

### From this visualization we can conclude that LSTM worked well. It can be improved though!

In [ ]:
!pip install see_rnn


In [ ]:
from see_rnn import *

In [ ]:
grads1 = get_gradients(lstm_model, 1, x_train_data, y_train_data)
grads2 = get_gradients(lstm_model,2,x_train_data,y_train_data)
output = get_outputs(lstm_model, 1, x_train_data)

# A Visual Exploration of LSTM's


## Layer 1 Visualization
Layer1 plots are as follows
1. 1D with 10 gradients
2. 1D with 500 gradients
3. 2D with 500 gradients
3. 1D with all gradients

In [ ]:
features_1D(grads1[:10] ,n_rows=2)
features_1D(grads1[:500] ,n_rows=2)
features_2D(grads1[:500])
features_1D(grads1 ,n_rows=2)


## Layer2 Visualization
Layer2 plots are as follows
1. 1D with 500 gradients
2. 2D with 500 gradients
3. 1D with all gradients
4. 2D with all gradients

In [ ]:
features_1D(grads2[:500], n_rows=2)
features_2D(grads2[:500], n_rows=2)
features_1D(grads2, n_rows=2)

## Output Layer Visualization

In [ ]:
features_1D(output, n_rows=2)
features_2D(output, n_rows=2)


## LSTM Histograms

In [ ]:
rnn_histogram(lstm_model, 'lstm', equate_axes=False)

## HEATMAP

In [ ]:
rnn_heatmap(lstm_model, 'lstm')